#### Prerequisites


In [4]:
from typing import List
from collections import defaultdict

## 992. Subarrays with K Different Integers

    Difficulty - Hard
    Topics - Array
    Algos - Hash Table, Sliding Window

Given an integer array nums and an integer k, return the number of good subarrays of nums.

A good array is an array where the number of different integers in that array is exactly k.

-   For example, [1,2,3,1,2] has 3 different integers: 1, 2, and 3.

A subarray is a contiguous part of an array.


In [5]:
class Solution:
    def subarraysWithAtMostKDistinct(self, nums: list[int], k: int) -> int:
        memo = defaultdict(int)
        ans = left = 0

        for right, num in enumerate(nums):
            memo[num] += 1
            while len(memo) > k:
                memo[nums[left]] -= 1
                if memo[nums[left]] == 0:
                    del memo[nums[left]]
                left += 1
            ans += right - left + 1

        return ans

    def subarraysWithKDistinct(self, nums: list[int], k: int) -> int:
        return self.subarraysWithAtMostKDistinct(
            nums, k
        ) - self.subarraysWithAtMostKDistinct(nums, k - 1)


if __name__ == "__main__":
    sol = Solution()
    cases = [{"nums": [1, 2, 1, 2, 3], "k": 2}, {"nums": [1, 2, 1, 3, 4], "k": 3}]
    for case in cases:
        print(sol.subarraysWithKDistinct(case["nums"], case["k"]))

7
3
